In [1]:
!pip install torchaudio encodec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45759 sh

In [3]:
import torchaudio
import torch
from encodec import EncodecModel
from encodec.utils import convert_audio

# Загрузка модели (24 kHz, 1.5 kbps)
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(1.5)

# Загрузка аудио (поддержка: WAV, MP3)
waveform, sample_rate = torchaudio.load("/content/sample-3s (1).wav")
waveform = convert_audio(waveform, sample_rate, model.sample_rate, model.channels)

# Кодирование в токены
with torch.no_grad():
    encoded_frames = model.encode(waveform.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

print("Токены:", codes)  # Например, tensor([[1, 2, 5, ...]])

Токены: tensor([[[  71,  678,  789,  905,    7,  640,  140,  281,  281,  789,  789,
          1008,  586,  735,  611,  845,    7,  330,  905,  968, 1004,  268,
           789,  281,  974,  492, 1008,  586,  573,  627,  249,  137,  637,
           607,  108,   72,  764,  137,  958,  281,  789,  386,  586,  843,
           492,  364,  624,  640,  394,  758,    7,  541,  281,  740,  185,
           586,  586,  982,  559,  806,  178,  607,  299,  541,  409,  135,
           137,  540,  789,  192,  499,  611,  499,  627,  806,  277,   13,
           492,  640,  611,  640,  586,  627,  611,  611,  192,  428,  764,
           428,  708,  525,  711,  367,  207,    7,  640,  774,  586,  499,
           627,  627,  192,  843,  627,  137,  364,  383,  541,  394,  541,
           343,    7,  627,  789,  666,  789,  428,  409,  611,  627,  499,
           611,  627,  499,  409,  461,  492,  627,  843,  192,  492,  495,
           611,  843,  492,  627,  627,  627,  627,  409,  409,  492,  492,
    

In [4]:
# Восстановление аудио из токенов
with torch.no_grad():
    decoded_waveform = model.decode([(codes, None)])

# Сохранение результата
torchaudio.save("output.wav", decoded_waveform.squeeze(0), model.sample_rate)